# M1.5 - Données d'observation de la Terre

*Partie de:* [**Open Climate Data**](https://github.com/OpenClimateScience/M1-Open-Climate-Data) | **Leçon précédente** | **Leçon suivante**

**Contenu :**

- [Sources de données satellitaires](#Sources-de-données-satellitaires)
- [Jeux de données multi-capteurs](#Jeux-de-données-multi-capteurs)
- [Utilisation des données satellitaires sur les précipitations](#Utilisation-des-données-satellitaires-sur-les-précipitations)
- [Zoom sur une zone d'étude](#Zoom-sur-une-zone-d'étude)
- [Utilisation de `cartopy`](#Utilisation-de-cartopy)
- [Fusion de plusieurs jeux de données de séries temporelles](#Fusion-de-plusieurs-jeux-de-données-de-séries-temporelles)

## Sources de données satellitaires

La constellation de satellites d'observation de la Terre de la NASA offre de nombreuses sources d'informations sur le système climatique terrestre. Le site [Eyes on the Earth de la NASA](https://eyes.nasa.gov/apps/earth/) montre où se trouvent de nombreux satellites en temps réel.

La plupart des satellites d'observation de la Terre de la NASA ont des **orbites polaires héliosynchrones** (voir la figure ci-dessous). Cela signifie que l'orbite du satellite passe toujours (presque) au-dessus des pôles terrestres tandis que la Terre tourne sous lui. Lorsque le satellite se déplace entre les pôles, il traverse l'équateur à la même heure locale, mais sur une partie différente de la Terre.

![](./assets/sun-synchronous.png)

*Image provenant de [NASA's Earth Observatory.](https://earthobservatory.nasa.gov/features/OrbitsCatalog)*

Les satellites d'observation de la Terre transportent des capteurs pointés vers la Terre pour effectuer des mesures de la surface ou de l'atmosphère. Les capteurs mesurent une partie du spectre électromagnétique : soit la lumière visible, l'infrarouge proche et infrarouge à ondes courtes, les ondes infrarouges ou les micro-ondes. Ces capteurs peuvent être décrits en termes de :

- **Résolution spectrale :** Le nombre de types d'énergie électromagnétique détectés et la précision de l'espacement entre les bandes de longueurs d'onde.
- **Résolution spatiale :** La plus petite taille de cible mesurable sur le sol.
- **Résolution temporelle :** La fréquence à laquelle un capteur acquiert des données sur une localisation spécifique.

La résolution temporelle est également appelée **temps de revisite :** le temps nécessaire pour que le capteur observe la même localisation sur la Terre sous le même angle de vue. Le temps de revisite détermine la fréquence à laquelle nous pouvons obtenir des informations sur les systèmes agricoles.

## Jeux de données multi-capteurs

Différentes missions satellitaires, qu'elles se chevauchent ou soient séparées par des années, mesurent parfois la même chose. **Les précipitations** sont une variable climatique importante pour l'agriculture, mesurée par plusieurs missions satellitaires dans la constellation de mesure des précipitations mondiales (GPM) de la NASA.

![](./assets/GPM-constellation.jpg)

### Données de précipitations IMERG

L'algorithme IMERG (Integrated Multi-satellite Retrievals for GPM) combine les données de ces différentes missions GPM pour estimer les précipitations totales à travers le globe à des intervalles de 30 minutes et avec une résolution de 10 km. Il existe trois (3) produits IMERG différents, différenciés par la manière dont ils intègrent les données et leur latence, c'est-à-dire la rapidité avec laquelle ils sont disponibles :

- IMERG "Early" a la latence la plus faible, disponible environ 4 heures après la collecte des données, mais peut être le moins précis car il ne projette que vers l'avant dans le temps.
- IMERG "Late" a une latence d'environ 14 heures et utilise des projections avant et arrière pour améliorer les estimations.
- IMERG "Final" a une latence d'environ 3,5 mois mais est le produit le plus précis car il intègre des données de pluviomètres au sol.

## Utilisation des données satellitaires sur les précipitations

Des parties du nord de l'Algérie et de la Tunisie ont connu des inondations soudaines en mai 2023. Utilisons le produit IMERG-Late pour quantifier la quantité totale de précipitations tombée dans la région sur une journée.


In [ ]:
import earthaccess
import xarray as xr
from matplotlib import pyplot

auth = earthaccess.login()

Bien que les données IMERG-Final soient produites à des intervalles de 30 minutes, nous utiliserons aujourd'hui [une version des données qui a été agrégée à des pas de temps quotidiens.](https://disc.gsfc.nasa.gov/datasets/GPM_3IMERGDF_06/summary)

In [ ]:
results = earthaccess.search_data(
    short_name = 'GPM_3IMERGDF',
    temporal = ('2023-05-25', '2023-05-30'))

results[0]

In [ ]:
# REMARQUE : open() nécessite une séquence de références de fichiers
files = earthaccess.open(results)
files

In [ ]:
ds = xr.open_dataset(files[1])
ds

In [ ]:
list(ds.variables.keys())

Il y a beaucoup de variables différentes dans ce jeu de données, toutes décrites dans [la documentation IMERG-Late.](https://disc.gsfc.nasa.gov/datasets/GPM_3IMERGDF_06/summary) Nous utiliserons la variable `precipitation`, qui est l'estimation multi-satellite, calibrée par pluviomètre, que la NASA recommande pour une utilisation générale.

In [ ]:
ds['precipitation']

In [ ]:
# REMARQUE : vmax = 100 facilite la visualisation des valeurs de précipitation plus faibles
ds['precipitation'].plot(vmax = 100)

Ça a l'air bizarre ! Des idées sur ce qui ne va pas ?

#### &#x1F6A9; <span style="color:red">Attention</red>

Nous devons faire pivoter le graphique pour que les lignes de l'image correspondent aux bandes de latitude. **En particulier, nous devons dire à `xarray` que la longitude ("lon") doit s'étendre sur l'axe X et que la latitude ("lat") doit s'étendre sur l'axe Y.**

In [ ]:
ds['precipitation'].plot(x = 'lon', y = 'lat', vmax = 100)

Cela semble mieux. Mais pourquoi la majeure partie de l'image est-elle sombre ?

Le moyen le plus simple de voir plus de détails dans l'image est de demander à `xarray` d'étirer la barre de couleur afin que les valeurs extrêmes ne dominent pas ; nous faisons cela avec `robust = True`.

In [ ]:
ds['precipitation'].plot(x = 'lon', y = 'lat', robust = True)

Nous pourrions également indiquer à `xarray` quelle doit être la valeur maximale assignée à une couleur, en utilisant l'argument `vmax`. Il y a un argument correspondant `vmin` pour la valeur minimale.

In [ ]:
ds['precipitation'].plot(x = 'lon', y = 'lat', vmax = 15)

## Zoom sur une zone d'étude

Comment pouvons-nous utiliser ces données pour des applications locales ? Nous devons trouver un moyen de concentrer la carte sur une zone plus petite.

La fonction `slice()` intégrée à Python peut être utilisée avec la méthode `sel()` d'un DataArray `xarray` pour découper un tableau plus grand en un tableau plus petit. Ici, nous nous concentrons sur une petite zone rectangulaire incluant Alger.

In [ ]:
# Zone entre 30 et 40 degrés N de latitude et entre 6 W et 6 E de longitude
precip = ds['precipitation'].sel(lat = slice(30, 40), lon = slice(-6, 6))

precip.plot(x = 'lon', y = 'lat')

## Utilisation de `cartopy`

Il est judicieux de vérifier que nous cartographions la bonne partie du monde, d'autant plus que les données IMERG sont rotées. `cartopy` est une bibliothèque Python qui fournit des outils cartographiques supplémentaires.

Ci-dessous, j'ai changé la carte des couleurs, `cmap`, afin qu'il soit plus facile de voir les côtes sombres au-dessus des données de précipitations.

Notez que Plate-Carree est simplement un terme élégant pour un système de coordonnées équirectangulaires, c'est-à-dire un tracé longitude-latitude.

In [ ]:
from cartopy import crs

proj = crs.PlateCarree()
style = {
    'projection': proj
}

plot = precip.plot(subplot_kws = style, transform = proj, cmap = 'magma_r', x = 'lon', y = 'lat')
plot.axes.coastlines()

[Vous pouvez voir les autres cartes de couleurs disponibles ici.](https://matplotlib.org/stable/gallery/color/colormap_reference.html) Toute carte de couleurs peut être inversée en ajoutant `'_r'` à la fin du nom de la carte des couleurs.

#### &#x1F3AF; Meilleure Pratique

**Il est important de choisir la bonne carte des couleurs pour vos données.** Les gens pensent parfois qu'une échelle de couleurs arc-en-ciel est meilleure car elle contient "plus de couleurs". Cependant, dans l'exemple ci-dessous, vous pouvez voir qu'une échelle de couleurs arc-en-ciel met en évidence différentes parties de l'échelle linéaire. Le changement de couleur entre 40 et 60 mm semble beaucoup plus marqué que le changement entre 0 et 20 mm, bien que ce soit la même taille d'étape (différence de 20 mm). C'est un exemple de la façon dont l'échelle de couleurs arc-en-ciel ne parvient pas à fournir une **linéarité perceptive** ; la perception d'un changement de teinte ou de luminosité qui est proportionnel au changement de valeur numérique (par exemple, précipitation).

In [ ]:
plot = precip.plot(subplot_kws = style, transform = proj, cmap = 'jet', x = 'lon', y = 'lat')
plot.axes.coastlines()

**Mais les échelles de couleurs arc-en-ciel posent également problème pour les personnes daltoniennes, comme vous pouvez le voir dans la simulation de ce que certains spectateurs daltoniens expérimenteraient en regardant différentes échelles de couleurs.**

![](assets/M1_fig_colorblind_scales.jpg)

*Image provenant de [Light & Bartlein (2004)](https://eos.org/features/the-end-of-the-rainbow-color-schemes-for-improved-data-graphics)*

**Heureusement, la plupart des cartes des couleurs disponibles dans `xarray` et `matplotlib` sont perceptuellement linéaires, et beaucoup d'entre elles restent cohérentes pour les spectateurs daltoniens.** Vous pouvez voir certaines de ces cartes des couleurs plus en détail sur le [site web du Dr Cynthia Brewer.](https://colorbrewer2.org/#type=sequential&scheme=BuGn&n=3)

**Alors, combien de pluie est tombée autour d'Alger ce jour-là ?**

In [ ]:
precip.sel(lon = 3.059, lat = 36.754, method = 'nearest').values

C'est une bonne quantité de pluie pour cette région en une seule journée, mais la carte ci-dessus montre clairement qu'il y a des zones près d'Alger qui ont reçu plus de pluie. Quelle est la précipitation maximale sur la côte d'Alger ?

In [ ]:
precip.sel(lon = slice(3, 3.2), lat = slice(36.5, 36.8)).max()

Petit rappel, étant donné que les `values` sont renvoyées sous forme de tableau NumPy, nous pouvons faire des calculs sur ces tableaux, les traiter comme s'il s'agissait de simples nombres. Ainsi, une conversion de mm à mètres est facile :

In [ ]:
precip.sel(lon = slice(3, 3.2), lat = slice(36.5, 36.8)).max() / 1000

## Fusion de plusieurs jeux de données de séries temporelles

Chaque granule IMERG de cette collection est une estimation quotidienne des précipitations.

In [ ]:
files

Plus tôt, nous avons développé une série temporelle de températures en ouvrant chaque fichier MERRA-2 dans une boucle `for`. Nous pouvons obtenir une série temporelle de précipitations de la même manière.

In [ ]:
# Cela peut prendre une demi-minute
datasets = []
for filename in files:
    ds = xr.open_dataset(filename)
    datasets.append(ds['precipitation'])

In [ ]:
len(datasets)

Dans l'exemple MERRA-2, nous avons extrait la valeur de température à un emplacement spécifique avant de l'ajouter à une liste. Mais `xarray` est capable de représenter plusieurs pas de temps dans un seul jeu de données. Est-il possible de fusionner les pas de temps adjacents ?

Nous pouvons faire cela avec la fonction `concat()` dans `xarray`. Nous spécifions que les multiples jeux de données que nous avons ouverts doivent être joints le long de la dimension `'time'`.

In [ ]:
time_series = xr.concat(datasets, dim = 'time')

Maintenant que nous avons tous les pas de temps dans un seul jeu de données, lorsque nous utilisons l'indexation basée sur des labels avec la fonction `sel()`, nous pouvons obtenir une estimation ponctuelle pour chaque pas de temps (6 jours au total).

In [ ]:
time_series.sel(lon = 3.059, lat = 36.754, method = 'nearest')

In [ ]:
time_series.sel(lon = 3.059, lat = 36.754, method = 'nearest').plot()